In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import bs4
import requests
from pprint import pprint
import nltk
import Memory_Collaborative_Filtering as mem

In [ ]:
def url_builder(book_title):
    path = 'https://isbndb.com/search/books/'
    title_list = book_title.split()
    final_path_list=[]
    for x in range(len(title_list)):
        if x==len(title_list)-1:
            final_path_list.append(title_list[x])
        else:
            final_path_list.append(title_list[x]+'%20')
            
    return path+''.join(final_path_list)

In [ ]:
def retrieve_page_content(url):
    page = requests.get(url)
    return page.content

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, bs4.element.Comment):
        return False
    return True


def text_from_html(body):
    soup = bs(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [ ]:
def retrieve_author(book_title, product_id, html_text, index):
    author='Cannot Locate'
    html_text_list = html_text.split()
    for x in range(len(html_text_list)):
        if html_text_list[x]=='Authors:':
            author=html_text_list[x+1]+' '+html_text_list[x+2]
            break
    author_dict = {'book_title': book_title, 'product_id': product_id, 'author': author}
    author_table = pd.DataFrame(author_dict, index=[index])
    return author_table

In [1]:
def main(data):
    final_authors=pd.DataFrame()
    counter=0
    for x in range(len(data)):
        counter+=1
        url=url_builder(data['product_title'][x])
        print(url)
        html=retrieve_page_content(url)
        text_html=text_from_html(html)
        author = retrieve_author(data['product_title'][x], data['product_id'][x], text_html, counter)
        final_authors = final_authors.append(author)
    return final_authors

In [ ]:
worm_path = 'C:/Users/arjun/Desktop/amazon.db'
review_data = mem.import_standard_data(worm_path)

In [ ]:
products = review_data[['product_id', 'product_title']]
products = products.drop_duplicates()
products = products.reset_index(drop=True)